# 0 - Quick Imports

In [1]:
# set path to the parent dir
import os
import sys
sys.path.append('src')
os.chdir(os.path.join(os.path.expanduser('~'),'project/r2e/r2e-edits-internal'))

!pwd

/data/home/jaskirats/project/r2e/r2e-edits-internal


In [2]:
import numpy as np
from collections import deque
import pandas as pd
import os,sys

import sys 
# sys.path.append(".")
# sys.path.append("..")

import random
# from torchvision import transforms, utils
from PIL import Image, ImageFilter 

import PIL
from PIL import Image
import numpy as np

# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

# seaborn
import seaborn as sns
sns.set()

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
def display_alongside_batch(img_list, resize_dims=(512,512)):
    if isinstance(resize_dims, int):
        resize_dims = (resize_dims,resize_dims)
    res = np.concatenate([np.array(img.resize(resize_dims)) for img in img_list], axis=1)
    return Image.fromarray(res)

In [4]:
from IPython.core.magic import register_cell_magic
from IPython.display import Markdown

@register_cell_magic
def info(line, cell):
    display(Markdown(f'<div class="alert alert-info"><strong>Info!</strong> {cell}</div>'))

@register_cell_magic
def warning(line, cell):
    display(Markdown(f'<div class="alert alert-warning"><strong>Warning!</strong> {cell}</div>'))

@register_cell_magic
def error(line, cell):
    display(Markdown(f'<div class="alert alert-danger"><strong>Error!</strong> {cell}</div>'))

from IPython.display import Markdown

def create_alert_block(alert_type, content):
    html = f'<div class="alert alert-{alert_type}"><strong>{alert_type.capitalize()}!</strong> {content}</div>'
    display(Markdown(html))

# 1 - Trajectory Collection

In [5]:
# from src.r2e_edits.agenthub.runtime.docker import DockerRuntime
# from src.r2e_edits.agenthub.environment.env import EnvArgs, RepoEnv
# from src.r2e_edits.agenthub.agent.agent import AgentArgs, Agent

# from docker_bash_utils.docker_list_tags import fetch_docker_tags
# from src.r2e_edits.agenthub.utils.log import get_logger
from pathlib import Path
from datasets import load_dataset

In [6]:
from src.r2e_edits.agenthub.utils.log import get_logger
logger = get_logger(name='jupyter')
logger.info('hello')

[03/01/25 23:04:53] INFO     hello

## 1.0 - Data Analysis - r2e edit dockers and swebench verified

In [7]:
r2e_ds = load_dataset("r2e-edits/r2e-dockers-v1")
r2e_ds

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'docker_image', 'commit_hash', 'parsed_commit_content', 'execution_result_content', 'modified_files', 'modified_entity_summaries', 'relevant_files', 'num_non_test_files', 'num_non_test_func_methods', 'num_non_test_lines', 'prompt', 'problem_statement', 'expected_output_json'],
        num_rows: 4578
    })
    dev_10pr_v1: Dataset({
        features: ['repo_name', 'docker_image', 'commit_hash', 'parsed_commit_content', 'execution_result_content', 'modified_files', 'modified_entity_summaries', 'relevant_files', 'num_non_test_files', 'num_non_test_func_methods', 'num_non_test_lines', 'prompt', 'problem_statement', 'expected_output_json'],
        num_rows: 100
    })
    dev_100pr_v1: Dataset({
        features: ['repo_name', 'docker_image', 'commit_hash', 'parsed_commit_content', 'execution_result_content', 'modified_files', 'modified_entity_summaries', 'relevant_files', 'num_non_test_files', 'num_non_test_func_methods',

In [8]:
swe_ds = load_dataset("r2e-edits/swebench-verified-v1")
swe_ds

DatasetDict({
    test: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit', 'parsed_commit', 'run_tests', 'docker_image'],
        num_rows: 500
    })
})

In [9]:
# select the dataset
# ds = swe_ds['test']
ds = r2e_ds['train']

# 1.1 - Load verifier data

In [38]:
df = load_dataset("r2e-edits/sonnet-end2end-r2e-dev_100pr_combined-maxstep40-v1")
df

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 3231
    })
})

In [48]:
swe_verifier_traj_ds = load_dataset("r2e-edits/32b_swebv_temp08_10_patch_verifier")
swe_verifier_traj_ds

DatasetDict({
    train: Dataset({
        features: ['messages', 'docker_images', 'rewards', 'exp_names', 'num_steps', 'patch_size', 'p2p_rate'],
        num_rows: 4710
    })
})

In [8]:
df = swe_verifier_traj_ds['train'].to_pandas()

In [9]:
df['p2p_rate'][12]

array(['test_am_pm (utils_tests.test_dateformat.DateFormatTests)',
       'test_date (utils_tests.test_dateformat.DateFormatTests)',
       'test_date_formats (utils_tests.test_dateformat.DateFormatTests)',
       'test_dateformat (utils_tests.test_dateformat.DateFormatTests)',
       'test_datetime_with_local_tzinfo (utils_tests.test_dateformat.DateFormatTests)',
       'test_datetime_with_tzinfo (utils_tests.test_dateformat.DateFormatTests)',
       'test_day_of_year_leap (utils_tests.test_dateformat.DateFormatTests)',
       'test_empty_format (utils_tests.test_dateformat.DateFormatTests)',
       'test_epoch (utils_tests.test_dateformat.DateFormatTests)',
       'test_futuredates (utils_tests.test_dateformat.DateFormatTests)',
       'test_invalid_time_format_specifiers (utils_tests.test_dateformat.DateFormatTests)',
       'test_microsecond (utils_tests.test_dateformat.DateFormatTests)',
       'test_naive_ambiguous_datetime (utils_tests.test_dateformat.DateFormatTests)',
       '

In [13]:
# visualize the outputs
swe_verifier_traj_ds['train'][0]['messages']

[{'content': 'You are an expert judge evaluating AI assistant interactions. Your task is to determine if the assistant successfully resolved the user\'s request.\n\nKey evaluation criteria:\n1. Did the assistant complete the main task requested by the user?\n2. Did the assistant handle all edge cases and requirements specified?\n3. Were there any errors or issues in the final solution?\n4. Did the assistant verify the solution works as intended?\n\nRespond only with "<judgement>YES</judgement>" or "<judgement>NO</judgement>".',
  'role': 'system'},
 {'content': 'Please evaluate the following interaction between an AI assistant and a user:\n\n=== INTERACTION LOG ===\n\n[SYSTEM]\nYou are a programming agent who is provided a github issue and repository bash environment and is tasked to solve certain tasks (e.g., file localization, testcase generation, code repair and editing etc) to resolve the issue.\n\nWe have access to the following functions:\n\n–– BEGIN FUNCTION #1: file_editor ––\n

In [49]:
logger.info(swe_verifier_traj_ds['train'][4200]['messages'][0]['content'])

[03/01/25 23:53:06] INFO     You are an expert judge evaluating AI assistant interactions. Your task is to         
                             determine if the assistant successfully resolved the user's request.                  
                                                                                                                   
                             Key evaluation criteria:                                                              
                             1. Did the assistant complete the main task requested by the user?                    
                             2. Did the assistant handle all edge cases and requirements specified?                
                             3. Were there any errors or issues in the final solution?                             
                             4. Did the assistant verify the solution works as intended?                           
                                                                                                                   
                             Respond only with "<judgement>YES</judgement>" or "<judgement>NO</judgement>".

In [52]:
logger.info(swe_verifier_traj_ds['train'][4202]['messages'][1]['content'])

[03/01/25 23:53:58] INFO     Please evaluate the following interaction between an AI assistant and a user:         
                                                                                                                   
                             === INTERACTION LOG ===                                                               
                                                                                                                   
                             [SYSTEM]                                                                              
                             You are a programming agent who is provided a github issue and repository bash        
                             environment and is tasked to solve certain tasks (e.g., file localization, testcase   
                             generation, code repair and editing etc) to resolve the issue.                        
                                                                                                                   
                             We have access to the following functions:                                            
                                                                                                                   
                             –– BEGIN FUNCTION #1: file_editor ––                                                  
                             Description:                                                                          
                             Custom editing tool for viewing, creating and editing files                           
                               •     State is persistent across command calls and discussions with the user        
                               •     If path is a file, view displays the result of applying cat -n. If path is a  
                             directory, view lists non-hidden files and directories up to 2 levels deep            
                               •     The create command cannot be used if the specified path already exists as a   
                             file                                                                                  
                               •     If a command generates a long output, it will be truncated and marked with    
                             <response clipped>                                                                    
                               •     The undo_edit command will revert the last edit made to the file at path      
                                                                                                                   
                             Notes for using the str_replace command:                                              
                               •     The old_str parameter should match EXACTLY one or more consecutive lines from 
                             the original file. Be mindful of whitespaces!                                         
                               •     If the old_str parameter is not unique in the file, the replacement will not  
                             be performed. Make sure to include enough context in old_str to make it unique        
                               •     The new_str parameter should contain the edited lines that should replace the 
                             old_str                                                                               
                                                                                                                   
                             Parameters:                                                                           
                               1.    command (string, required)                                                    
                             Allowed values: [view, create, str_replace, insert, undo_edit]                        
                             The command to run.        

In [16]:
logger.info(swe_verifier_traj_ds['train'][40]['messages'][2]['content'])

[02/28/25 23:55:05] INFO     <judgement>NO</judgement>

In [30]:
import re
import litellm
from typing import Any, Dict, List, Optional, Tuple

def _count_tokens(messages: List[Dict[str, str]], llm_name='gpt4o') -> int:
        """
        Counts the tokens for a list of messages using the litellm library.
        Adjust as needed depending on the model and library.
        """
        token_count = litellm.token_counter(model=llm_name, messages=messages)
        print(f"Total tokens in conversation: {token_count}")
        return token_count
    
def dummy_count_tokens(s: str) -> int:
    """
    Dummy token counter that counts words.
    Replace this with your actual token counting logic.
    """
    messages = [
        {'role': 'system', 'content': "Identify whether the following agent trajectory is correct or not. Answer 'YES' or 'NO'"},
        {'role': 'user', 'content': s}
    ]
    
    return _count_tokens(messages)

def condense(input_str: str, max_tokens: int = 28000) -> str:
    """
    If the token count of input_str exceeds max_tokens, then starting with the second
    [USER]...[/USER] block (the oldest after the first), replace its inner content with
    a placeholder until the total token count is under the limit.
    
    The first [USER] block is left intact.
    """
    placeholder = "<Observation condensed for saving context>"
    
    # Check initial token count
    if dummy_count_tokens(input_str) <= max_tokens:
        return input_str

    # Regex to match [USER] blocks
    pattern = re.compile(r'(\[USER\])(.*?)(\[/USER\])', re.DOTALL)
    
    new_str = input_str
    # Continue condensing until token count is below the threshold or nothing changes.
    while dummy_count_tokens(new_str) > max_tokens:
        # Re-find all [USER] blocks in the updated string
        matches = list(pattern.finditer(new_str))
        if len(matches) <= 1:
            # i.e. --> either no [USER] blocks or only one exists
            break

        replaced = False
        # Iterate over all [USER] blocks starting from the second one
        for i, m in enumerate(matches):
            if i == 0:
                continue  # leave the first [USER] block unchanged
            # If already condensed, skip it
            if m.group(2).strip() == placeholder:
                continue
            # Build the new block with condensed content
            new_block = m.group(1) + placeholder + m.group(3)
            # Replace this block in the string using its current indices
            start, end = m.start(), m.end()
            new_str = new_str[:start] + new_block + new_str[end:]
            replaced = True
            # Break out after replacing one block so we can re-check token count
            break
        if not replaced:
            # All subsequent [USER] blocks are already condensed
            break

    return new_str

In [31]:
s = swe_verifier_traj_ds['train'][0]['messages'][1]['content']
out = condense(input_str = s, max_tokens = 4000)

Total tokens in conversation: 10914
Total tokens in conversation: 10914
Total tokens in conversation: 10187
Total tokens in conversation: 9269
Total tokens in conversation: 8745
Total tokens in conversation: 8733
Total tokens in conversation: 5960
Total tokens in conversation: 5852
Total tokens in conversation: 5559
Total tokens in conversation: 5172
Total tokens in conversation: 4968
Total tokens in conversation: 4312
Total tokens in conversation: 4173
Total tokens in conversation: 4161
Total tokens in conversation: 4164


In [32]:
logger.info(out)

[02/26/25 05:14:14] INFO     Please evaluate the following interaction between an AI assistant and a user:         
                                                                                                                   
                             === INTERACTION LOG ===                                                               
                                                                                                                   
                             [SYSTEM]                                                                              
                             You are a programming agent who is provided a github issue and repository bash        
                             environment and is tasked to solve certain tasks (e.g., file localization, testcase   
                             generation, code repair and editing etc) to resolve the issue.                        
                                                                                                                   
                             We have access to the following functions:                                            
                                                                                                                   
                             –– BEGIN FUNCTION #1: file_editor ––                                                  
                             Description:                                                                          
                             Custom editing tool for viewing, creating and editing files                           
                               •     State is persistent across command calls and discussions with the user        
                               •     If path is a file, view displays the result of applying cat -n. If path is a  
                             directory, view lists non-hidden files and directories up to 2 levels deep            
                               •     The create command cannot be used if the specified path already exists as a   
                             file                                                                                  
                               •     If a command generates a long output, it will be truncated and marked with    
                             <response clipped>                                                                    
                               •     The undo_edit command will revert the last edit made to the file at path      
                                                                                                                   
                             Notes for using the str_replace command:                                              
                               •     The old_str parameter should match EXACTLY one or more consecutive lines from 
                             the original file. Be mindful of whitespaces!                                         
                               •     If the old_str parameter is not unique in the file, the replacement will not  
                             be performed. Make sure to include enough context in old_str to make it unique        
                               •     The new_str parameter should contain the edited lines that should replace the 
                             old_str                                                                               
                                                                                                                   
                             Parameters:                                                                           
                               1.    command (string, required)                                                    
                             Allowed values: [view, create, str_replace, insert, undo_edit]                        
                             The command to run.        

In [37]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("./src/r2e_edits/agenthub/train/results.csv")
df = pd.read_csv("./src/r2e_edits/agenthub/train/results")

# Group by 'docker_images' and select the row with the maximum avg_yes_prob from each group
aggregated = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["avg_yes_prob"].idxmax()]
).reset_index(drop=True)

# Calculate the overall mean of 'gt_correct'
overall_accuracy = aggregated["gt_correct"].mean()
print ("total number of trajs:", len(df))
print("Overall Accuracy:", overall_accuracy)

# if avg_yes_prob > 0.5, the predicted answer is YES (1), otherwise NO (0)
df['predicted'] = (df['avg_yes_prob'] > 0.5).astype(int)
# Ensure the ground truth is represented as 0 or 1 (if it's not already)
df['gt_correct'] = df['gt_correct'].astype(int)
# Create a column that shows if the prediction matches the ground truth for that entry
df['correct_prediction'] = (df['predicted'] == df['gt_correct']).astype(int)

# # Print per-entry details (you can adjust the columns as needed)
# print("Per-entry accuracy details:")
# print(df[['docker_images', 'avg_yes_prob', 'gt_correct', 'predicted', 'correct_prediction']])

# Calculate the overall average accuracy
overall_accuracy = df['correct_prediction'].mean()
print("\nOverall RM Accuracy:", overall_accuracy)

FileNotFoundError: [Errno 2] No such file or directory: './src/r2e_edits/agenthub/train/results'

In [40]:
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-32B_20k-v1.csv")
df

,docker_images,rewards,exp_names,num_steps,patch_size,p2p_rate,avg_yes_prob,gt_correct
0,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,19,711,['lib/matplotlib/tests/test_contour.py::test_c...,0.437823,False
1,slimshetty/swebench-verified:sweb.eval.x86_64....,1.0,32B_temp08_r8,34,1293,['test_args_kwargs_signature (migrations.test_...,0.679179,True
2,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,11,550,['test_get_random_secret_key (user_commands.te...,0.851953,False
3,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,22,0,['test_deconstruct (expressions.tests.FTests)'...,0.106691,False
4,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,13,563,['test_mathml_printer' 'test_content_printmeth...,0.437823,False
...,...,...,...,...,...,...,...,...
251,slimshetty/swebench-verified:sweb.eval.x86_64....,1.0,32B_temp08_r8,30,772,['Assigning ImageField to None clears dimensio...,0.851953,True
252,slimshetty/swebench-verified:sweb.eval.x86_64....,1.0,32B_temp08_r8,14,794,['testing/test_unittest.py::test_simple_unitte...,0.835484,True
253,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,29,892,['test_str (auth_tests.test_models.GroupTests)...,0.679179,False
254,slimshetty/swebench-verified:sweb.eval.x86_64....,1.0,32B_temp08_r8,22,1500,['sklearn/model_selection/tests/test_search.py...,0.679179,True


In [17]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load the CSV file into a DataFrame
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifier-14B-v2.csv")
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew-14B-v3.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew-32B_20k-v1.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew-32B_32k-v3.csv")
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B-v1.csv")
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-32B_20k-v1.csv")

# df = df[:5000]
def pdivide():
    print ("-"*50)

import pandas as pd

def accuracy_p2p_steps(all_df, N=500):
    # 1. For each docker image, get the maximum p2p_rate.
    highest_p2ps_by_docker = all_df.groupby("docker_images")["p2p_rate"].max()

    # 2. Merge back to get the rows with this highest p2p_rate.
    highest_p2ps_df = pd.merge(
        all_df, highest_p2ps_by_docker, on=["docker_images", "p2p_rate"]
    )

    # 3. Within those rows, for each docker image, find the minimum step_count.
    highest_p2ps_df_min_steps = highest_p2ps_df.groupby("docker_images")["num_steps"].min()

    # 4. Merge again to filter rows having both the highest p2p_rate and minimum step_count.
    highest_p2ps_df = pd.merge(
        highest_p2ps_df, highest_p2ps_df_min_steps, on=["docker_images", "num_steps"]
    )

    # 5. Group by docker_image and compute the mean pass@1 value, sum across groups, then normalize.
    pass1 = highest_p2ps_df.groupby("docker_images")["gt_correct"].mean().sum() / N
    return pass1

def accuracy_topkyesprob_then_p2p(df, k=3):
    """
    For each group (docker_images):
       - Create a new column 'p2p_rate_len' as the length of the list in 'p2p_rate'.
       - Sort by avg_yes_prob descending.
       - Take the top k rows.
       - From those, pick the row with the highest p2p_rate_len.
       - final_correct = that row's gt_correct (0 or 1).

    Overall accuracy = mean of final_correct across groups.
    """
    # Create a new column with the length of the p2p_rate list
    df = df.copy()  
    df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    grouped = df.groupby("docker_images")
    final_corrects = []

    for docker_image, group in grouped:
        # Sort by avg_yes_prob descending and take top k rows
        topk = group.sort_values(by="avg_yes_prob", ascending=False).head(k)

        # Among the top k, select the row with the highest p2p_rate_len
        chosen_row = topk.loc[topk["p2p_rate_len"].idxmax()]

        # Record this group's correctness (0 or 1)
        final_corrects.append(chosen_row["gt_correct"])

    # Overall accuracy is the mean of the final correctness values across groups
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / len(final_corrects)
    
def accuracy_topkp2p_then_yesprob(df, k=3):
    """
    For each group (docker_images):
       - Create a new column 'p2p_rate_len' as the length of the list in 'p2p_rate'.
       - Sort by 'p2p_rate_len' descending.
       - Take the top k rows.
       - From those, pick the row with the highest avg_yes_prob.
       - final_correct = that row's gt_correct (0 or 1).

    Overall accuracy = mean of final_correct across groups.
    """
    # Create a new column with the length of the p2p_rate list
    df = df.copy()  # To avoid modifying the original DataFrame
    df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    grouped = df.groupby("docker_images")
    final_corrects = []

    for docker_image, group in grouped:
        # Sort by p2p_rate_len descending and take the top k rows
        topk = group.sort_values(by="p2p_rate_len", ascending=False).head(k)
        
        # Among the top k, select the row with the highest avg_yes_prob
        chosen_row = topk.loc[topk["avg_yes_prob"].idxmax()]
        
        # Record this group's correctness (0 or 1)
        final_corrects.append(chosen_row["gt_correct"])

    # Overall accuracy is the mean of the final correctness values across groups
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / len(final_corrects)

def accuracy_topksteps_then_max_yesprob(df, k=3):
    """
    For each group (docker_images):
       - Create a new column 'p2p_rate_len' as the length of the list in 'p2p_rate'.
       - Sort by 'p2p_rate_len' descending.
       - Take the top k rows.
       - From those, pick the row with the highest avg_yes_prob.
       - final_correct = that row's gt_correct (0 or 1).
       
    Overall accuracy is computed as a weighted average, where each group's 
    prediction is weighted by the chosen row's 'p2p_rate_len'.
    """
    # Create a new column with the length of the p2p_rate list
    # df = df.copy()  # To avoid modifying the original DataFrame
    # df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    grouped = df.groupby("docker_images")
    weighted_sum = 0.0
    total_weight = 0.0

    for docker_image, group in grouped:
        # Sort by the newly created 'p2p_rate_len' column descending
        group_sorted = group.sort_values(by="num_steps", ascending=True)
        
        # Take the top k rows based on the length of p2p_rate
        topk = group_sorted.head(k)
        
        # Among these, pick the row with the highest avg_yes_prob
        chosen_row = topk.loc[topk["avg_yes_prob"].idxmax()]
        
        # Use the chosen row's p2p_rate_len as the weight
        weight = chosen_row["num_steps"]
        weighted_sum += chosen_row["gt_correct"] / weight
        total_weight += weight

    if total_weight == 0:
        return 0.0
    return weighted_sum / total_weight

    
# Group by 'docker_images' and select the row with the maximum avg_yes_prob from each group
aggregated = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["avg_yes_prob"].idxmax()]
).reset_index(drop=True)

# Calculate the overall mean of 'gt_correct'
overall_accuracy = aggregated["gt_correct"].mean()
print ("total number of trajs:", len(df))
print("Overall Accuracy:", overall_accuracy)
pdivide()

# approach #2: Top 3 by avg_yes_prob, pick highest p2p_rate
k=5
acc_topk = accuracy_topk_then_max_p2p(df,k=k)
print(f"Accuracy (top k [{k}] yesprob -> highest p2p_rate):", acc_topk)

acc_topkp2p_then_max_yesprob = accuracy_topkp2p_then_max_yesprob(df, k=k)
print(f"Accuracy (top k [{k}] p2p -> highest yesprob):", acc_topkp2p_then_max_yesprob)

acc_p2p_steps = accuracy_p2p_steps(df)
print("Accuracy (p2p -> steps):", acc_p2p_steps)

pdivide()

# Aggregation using maximum avg_yes_prob
aggregated_avg = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["avg_yes_prob"].idxmax()]
).reset_index(drop=True)
overall_accuracy_avg = aggregated_avg["gt_correct"].mean()

# Aggregation using minimum num_steps
aggregated_min_steps = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["num_steps"].idxmin()]
).reset_index(drop=True)
overall_accuracy_min_steps = aggregated_min_steps["gt_correct"].mean()

# Aggregation using minimum patch_size
aggregated_min_patch = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["patch_size"].idxmin()]
).reset_index(drop=True)
overall_accuracy_min_patch = aggregated_min_patch["gt_correct"].mean()

# Aggregation using maximum p2p_rate
aggregated_max_p2p = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["p2p_rate"].idxmax()]
).reset_index(drop=True)
overall_accuracy_max_p2p = aggregated_max_p2p["gt_correct"].mean()

print("Total number of trajectories:", len(df))
print("Overall Accuracy (max avg_yes_prob):", overall_accuracy_avg)
print("Overall Accuracy (min num_steps):", overall_accuracy_min_steps)
print("Overall Accuracy (min patch_size):", overall_accuracy_min_patch)
print("Overall Accuracy (max p2p_rate):", overall_accuracy_max_p2p)
pdivide()

# compute pass@k
for k in range(3,4):
    pass_at_k = df.groupby("docker_images").apply(
        lambda group: group.sort_values("avg_yes_prob", ascending=False).head(k)["gt_correct"].any()
    ).mean()
    print(f"pass@{k}:", pass_at_k)

    k = 3
    def compute_margin(group, k=k):
        # Sort the group by avg_yes_prob in descending order
        sorted_probs = group.sort_values("avg_yes_prob", ascending=False)["avg_yes_prob"]
        # Only compute margin if we have at least k candidates
        if len(sorted_probs) < k:
            return None
        # Compute margin: difference between the top candidate and the kth candidate
        margin = sorted_probs.iloc[0] - sorted_probs.iloc[k-1]
        return margin
    
    # Apply the function to each docker_images group
    margins = df.groupby("docker_images").apply(compute_margin)
    
    print(f"Margin (difference) from top to top@{k} probability for each docker_images group:")
    print(margins)
    
    #compute and print the average margin across groups with at least k entries
    valid_margins = margins.dropna()
    if not valid_margins.empty:
        avg_margin = valid_margins.mean()
        print(f"Average margin from top to top@{k}: {avg_margin}")
pdivide()

# if avg_yes_prob > 0.5, the predicted answer is YES (1), otherwise NO (0)
df['predicted'] = (df['avg_yes_prob'] > 0.5).astype(int)
df['gt_correct'] = df['gt_correct'].astype(int)
df['correct_prediction'] = (df['predicted'] == df['gt_correct']).astype(int)

# Calculate the overall average accuracy
overall_accuracy = df['correct_prediction'].mean()
print("\nOverall RM Accuracy:", overall_accuracy)

# Compute descriptive statistics for avg_yes_prob
desc_stats = df['avg_yes_prob'].describe()
print("Descriptive statistics for avg_yes_prob:")
print(desc_stats)

# Plot the distribution using a histogram with a KDE overlay
plt.figure(figsize=(8, 6))
sns.histplot(df['avg_yes_prob'], bins=20, kde=True)
plt.title("Distribution of Avg Yes Probabilities")
plt.xlabel("Avg Yes Probability")
plt.ylabel("Frequency")
plt.show()

total number of trajs: 4710
Overall Accuracy: 0.39
--------------------------------------------------


NameError: name 'accuracy_topk_then_max_p2p' is not defined

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load the CSV file into a DataFrame
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifier-14B-v2.csv")
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew-14B-v3.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew-32B_20k-v1.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew-32B_32k-v3.csv")
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B-v1.csv")
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-32B_20k-v1.csv")

# df = df[:5000]
def pdivide():
    print ("-"*50)

import pandas as pd


def accuracy_topk_then_max_p2p(df, k=3):
    """
    For each group (docker_images):
       - Create a new column 'p2p_rate_len' as the length of the list in 'p2p_rate'.
       - Sort by avg_yes_prob descending.
       - Take the top k rows.
       - From those, pick the row with the highest p2p_rate_len.
       - final_correct = that row's gt_correct (0 or 1).

    Overall accuracy = mean of final_correct across groups.
    """
    # Create a new column with the length of the p2p_rate list
    df = df.copy()  
    df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    grouped = df.groupby("docker_images")
    final_corrects = []

    for docker_image, group in grouped:
        # Sort by avg_yes_prob descending and take top k rows
        topk = group.sort_values(by="avg_yes_prob", ascending=False).head(k)

        # Among the top k, select the row with the highest p2p_rate_len
        chosen_row = topk.loc[topk["p2p_rate_len"].idxmax()]

        # Record this group's correctness (0 or 1)
        final_corrects.append(chosen_row["gt_correct"])

    # Overall accuracy is the mean of the final correctness values across groups
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / len(final_corrects)
    
def accuracy_topkp2p_then_max_yesprob(df, k=3):
    """
    For each group (docker_images):
       - Create a new column 'p2p_rate_len' as the length of the list in 'p2p_rate'.
       - Sort by 'p2p_rate_len' descending.
       - Take the top k rows.
       - From those, pick the row with the highest avg_yes_prob.
       - final_correct = that row's gt_correct (0 or 1).

    Overall accuracy = mean of final_correct across groups.
    """
    # Create a new column with the length of the p2p_rate list
    df = df.copy()  # To avoid modifying the original DataFrame
    df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    grouped = df.groupby("docker_images")
    final_corrects = []

    for docker_image, group in grouped:
        # Sort by p2p_rate_len descending and take the top k rows
        topk = group.sort_values(by="p2p_rate_len", ascending=False).head(k)
        
        # Among the top k, select the row with the highest avg_yes_prob
        chosen_row = topk.loc[topk["avg_yes_prob"].idxmax()]
        
        # Record this group's correctness (0 or 1)
        final_corrects.append(chosen_row["gt_correct"])

    # Overall accuracy is the mean of the final correctness values across groups
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / len(final_corrects)

def accuracy_topksteps_then_max_yesprob(df, k=3):
    grouped = df.groupby("docker_images")
    weighted_sum = 0.0
    total_weight = 0.0

    for docker_image, group in grouped:
        # Sort by the newly created 'p2p_rate_len' column descending
        group_sorted = group.sort_values(by="num_steps", ascending=True)
        
        # Take the top k rows based on the length of p2p_rate
        topk = group_sorted.head(k)
        
        # Among these, pick the row with the highest avg_yes_prob
        chosen_row = topk.loc[topk["avg_yes_prob"].idxmax()]
        
        # Use the chosen row's p2p_rate_len as the weight
        weight = chosen_row["num_steps"]
        weighted_sum += chosen_row["gt_correct"] / weight
        total_weight += weight

    if total_weight == 0:
        return 0.0
    return weighted_sum / total_weight

    
# Group by 'docker_images' and select the row with the maximum avg_yes_prob from each group
aggregated = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["avg_yes_prob"].idxmax()]
).reset_index(drop=True)

# Calculate the overall mean of 'gt_correct'
overall_accuracy = aggregated["gt_correct"].mean()
print ("total number of trajs:", len(df))
print("Overall Accuracy:", overall_accuracy)
pdivide()

# approach #2: Top 3 by avg_yes_prob, pick highest p2p_rate
k=5
acc_topk = accuracy_topk_then_max_p2p(df,k=k)
print(f"Accuracy (top k [{k}] yesprob -> highest p2p_rate):", acc_topk)

acc_topkp2p_then_max_yesprob = accuracy_topkp2p_then_max_yesprob(df, k=k)
print(f"Accuracy (top k [{k}] p2p -> highest yesprob):", acc_topkp2p_then_max_yesprob)

acc_p2p_steps = accuracy_p2p_steps(df)
print("Accuracy (p2p -> steps):", acc_p2p_steps)

pdivide()


# compute pass@k
for k in range(3,4):
    pass_at_k = df.groupby("docker_images").apply(
        lambda group: group.sort_values("avg_yes_prob", ascending=False).head(k)["gt_correct"].any()
    ).mean()
    print(f"pass@{k}:", pass_at_k)

    k = 3
    def compute_margin(group, k=k):
        # Sort the group by avg_yes_prob in descending order
        sorted_probs = group.sort_values("avg_yes_prob", ascending=False)["avg_yes_prob"]
        # Only compute margin if we have at least k candidates
        if len(sorted_probs) < k:
            return None
        # Compute margin: difference between the top candidate and the kth candidate
        margin = sorted_probs.iloc[0] - sorted_probs.iloc[k-1]
        return margin
    
    # Apply the function to each docker_images group
    margins = df.groupby("docker_images").apply(compute_margin)
    
    print(f"Margin (difference) from top to top@{k} probability for each docker_images group:")
    print(margins)
    
    #compute and print the average margin across groups with at least k entries
    valid_margins = margins.dropna()
    if not valid_margins.empty:
        avg_margin = valid_margins.mean()
        print(f"Average margin from top to top@{k}: {avg_margin}")
pdivide()

# if avg_yes_prob > 0.5, the predicted answer is YES (1), otherwise NO (0)
df['predicted'] = (df['avg_yes_prob'] > 0.5).astype(int)
df['gt_correct'] = df['gt_correct'].astype(int)
df['correct_prediction'] = (df['predicted'] == df['gt_correct']).astype(int)

# Calculate the overall average accuracy
overall_accuracy = df['correct_prediction'].mean()
print("\nOverall RM Accuracy:", overall_accuracy)

# Compute descriptive statistics for avg_yes_prob
desc_stats = df['avg_yes_prob'].describe()
print("Descriptive statistics for avg_yes_prob:")
print(desc_stats)

# Plot the distribution using a histogram with a KDE overlay
plt.figure(figsize=(8, 6))
sns.histplot(df['avg_yes_prob'], bins=20, kde=True)
plt.title("Distribution of Avg Yes Probabilities")
plt.xlabel("Avg Yes Probability")
plt.ylabel("Frequency")
plt.show()

In [11]:
def accuracy_topkstepcount_then_yesprob(df, k=3):
    grouped = df.groupby("docker_images")
    final_corrects = []
    
    for docker_image, group in grouped:
        # Sort by step_count ascending and take top k
        # topk = group.sort_values(by="num_steps", ascending=True).head(k)
        topk = get_topk_with_ties(group, "num_steps", k, ascending=True)
        # Among the top k, select the row with the highest avg_yes_prob
        chosen_row = topk.loc[topk["avg_yes_prob"].idxmax()]
        final_corrects.append(chosen_row["gt_correct"])
    
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / len(final_corrects)

def accuracy_topkyesprob_then_stepcount(df, k=3):
    grouped = df.groupby("docker_images")
    final_corrects = []
    
    for docker_image, group in grouped:
        # Sort by avg_yes_prob descending and take top k
        topk = group.sort_values(by="avg_yes_prob", ascending=False).head(k)
        # Among the top k, select the row with the lowest step_count
        chosen_row = topk.loc[topk["num_steps"].idxmin()]
        final_corrects.append(chosen_row["gt_correct"])
    
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / len(final_corrects)

In [12]:
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-32B_20k-v1.csv")
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B_32k-v1.csv")

# Group by 'docker_images' and select the row with the maximum avg_yes_prob from each group
aggregated = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["avg_yes_prob"].idxmax()]
).reset_index(drop=True)

# Calculate the overall mean of 'gt_correct'
overall_accuracy = aggregated["gt_correct"].mean()
print ("total number of trajs:", len(df))
print("Overall Accuracy:", overall_accuracy)


k=2
acc_topk = accuracy_topkstepcount_then_yesprob(df,k=k)
print(f"Accuracy (top k [{k}] yesprob -> lowest step count):", acc_topk)

k=5
acc_topk = accuracy_topkyesprob_then_stepcount(df,k=k)
print(f"Accuracy (top k [{k}] stepcount -> highest yesprob):", acc_topk)

total number of trajs: 4710
Overall Accuracy: 0.408


/tmp/ipykernel_3222380/1434896223.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated = df.groupby("docker_images", as_index=False).apply(


NameError: name 'get_topk_with_ties' is not defined

In [13]:
N = 500
def accuracy_topkyesprob_then_p2p(df, k=3):
    df = df.copy()  
    df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    grouped = df.groupby("docker_images")
    final_corrects = []

    for docker_image, group in grouped:
        # Sort by avg_yes_prob descending and take top k rows
        topk = group.sort_values(by="avg_yes_prob", ascending=False).head(k)

        # Among the top k, select the row with the highest p2p_rate_len
        chosen_row = topk.loc[topk["p2p_rate_len"].idxmax()]
        final_corrects.append(chosen_row["gt_correct"])

    # Overall accuracy
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / N
    
def accuracy_topkp2p_then_yesprob(df, k=3):
    df = df.copy() 
    df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    grouped = df.groupby("docker_images")
    final_corrects = []

    for docker_image, group in grouped:
        # Sort by p2p_rate_len descending and take the top k rows
        # topk = group.sort_values(by="p2p_rate_len", ascending=False).head(k)
        topk = get_topk_with_ties(group, "p2p_rate_len", k)
        
        # Among the top k, select the row with the highest avg_yes_prob
        chosen_row = topk.loc[topk["avg_yes_prob"].idxmax()]
        
        final_corrects.append(chosen_row["gt_correct"])

    # Overall accuracy
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / N

def accuracy_topkp2p_then_yesprob_alt(df, k=3):
    df = df.copy()
    df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    # Helper function to get top-k rows with ties based on a given column.
    def get_topk_with_ties(group, col, k):
        group_sorted = group.sort_values(by=col, ascending=False)
        if len(group_sorted) <= k:
            return group_sorted
        kth_value = group_sorted.iloc[k-1][col]
        return group_sorted[group_sorted[col] >= kth_value]
    
    grouped = df.groupby("docker_images")
    final_corrects = []
    
    for docker_image, group in grouped:
        # Normalize num_steps for the group.
        steps_min = group['num_steps'].min()
        steps_max = group['num_steps'].max()
        if steps_max == steps_min:
            group['num_steps_norm'] = 0.0
        else:
            group['num_steps_norm'] = (group['num_steps'] - steps_min) / (steps_max - steps_min)
        
        # Compute the adjusted score.
        group['adjusted_score'] = group['avg_yes_prob'] #- group['num_steps_norm']
        
        # Select top k rows based on p2p_rate_len (keeping ties).
        topk = get_topk_with_ties(group, "p2p_rate_len", k)
        
        # Among these top rows, select the row with the highest adjusted_score.
        chosen_row = topk.loc[topk["adjusted_score"].idxmax()]
        final_corrects.append(chosen_row["gt_correct"])
    
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / len(final_corrects)
    
def get_topk_with_ties(group, key, k, ascending=False):
    # Sort the group by 'p2p_rate_len' in descending order.
    group_sorted = group.sort_values(by=key, ascending=ascending)
    if len(group_sorted) <= k:
        return group_sorted
    kth_value = group_sorted.iloc[k-1][key]
    # Keep all rows where p2p_rate_len is >= kth_value
    return group_sorted[group_sorted[key] >= kth_value]

In [153]:
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-32B_20k-v1.csv")
df1 = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B_32k-v1.csv")
df2 = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B-t06-v1.csv")
# df2 = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B-t06-v1.csv")
df3 = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B_32k-t0-v1.csv")
df1 = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B-32k-lora64-1en5-v2.csv")
df2 = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B-32k-t06-lora64-1en5-v1.csv")

# df = pd.concat([df1, df3], ignore_index=True)

# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B-32k-lora64-1en5-v1.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew_p2p-14B-32k-lora64-1en5-v2.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_qwen25coder-14b-instruct-verifier-sonnet_32b_gpt4o_combined_32k_verifiernew_reasoning_ep2-32k-lora64-lr1en4-v1-v1.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_qwen25coder-32b-instruct-verifier-sonnet_32b_gpt4o_combined_32k_verifiernew_reasoning_ep2-32k-lora64-v1-v1.csv")
# df= pd.read_csv("./src/r2e_edits/agenthub/train/results_qwen25coder-14b-instruct-verifier-sonnet_32b_gpt4o_combined_32k_verifiernew_reasoning_ep2-32k-lora64-v1-v1.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_qwen25coder-14b-instruct-verifier-sonnet_32b_gpt4o_combined_32k_verifiernew_reasoning_ep2-32k-lora64-lr1en3-v1-v1.csv")
# df = pd.read_csv("./src/r2e_edits/agenthub/train/results_qwen25coder-14b-instruct-verifier-sonnet_32b_gpt4o_combined_32k_verifiernew_reasoning_ep2-32k-lora64-lr5en6-v1-v1.csv")

# df = pd.concat([df1, df2, df3], ignore_index=True)
# df = df2

# df = df[:2688]
# Group by 'docker_images' and select the row with the maximum avg_yes_prob from each group
aggregated = df.groupby("docker_images", as_index=False).apply(
    lambda group: group.loc[group["avg_yes_prob"].idxmax()]
).reset_index(drop=True)

# Calculate the overall mean of 'gt_correct'
overall_accuracy = aggregated["gt_correct"].mean()
print ("total number of trajs:", len(df))
print("Overall Accuracy:", overall_accuracy)

k=5
acc_topk = accuracy_topkyesprob_then_p2p(df,k=k)
print(f"Accuracy (top k [{k}] yesprob -> highest p2p_rate):", acc_topk)

k=1
acc_topk = accuracy_topkp2p_then_yesprob(df,k=k)
print(f"Accuracy (top k [{k}] p2p -> highest yesprob):", acc_topk)

/tmp/ipykernel_3222380/271366871.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated = df.groupby("docker_images", as_index=False).apply(


total number of trajs: 10163
Overall Accuracy: 0.424
Accuracy (top k [5] yesprob -> highest p2p_rate): 0.452
Accuracy (top k [1] p2p -> highest yesprob): 0.452


In [146]:
divider = "-"*50
divider

'--------------------------------------------------'

In [101]:
df1

,docker_images,rewards,exp_names,num_steps,patch_size,p2p_rate,avg_yes_prob,gt_correct
0,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,33,1136,['test_modelchoicefield (forms_tests.tests.tes...,0.622459,False
1,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,10,716,['test_get_filesystem_storage (file_storage.te...,0.705785,False
2,slimshetty/swebench-verified:sweb.eval.x86_64....,1.0,32B_temp08_r8,17,931,['AlterField should optimize into AddField.'\n...,0.731059,True
3,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,22,0,['test_deconstruct (expressions.tests.FTests)'...,0.017986,False
4,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r8,13,563,['test_mathml_printer' 'test_content_printmeth...,0.377541,False
...,...,...,...,...,...,...,...,...
4705,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r2,40,3582,['test_naturals' 'test_naturals0' 'test_intege...,0.320821,False
4706,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r2,41,0,['test_ticket8597 (queries.tests.ComparisonTes...,0.001927,False
4707,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r2,31,1648,['astropy/io/fits/tests/test_header.py::test_s...,0.348645,False
4708,slimshetty/swebench-verified:sweb.eval.x86_64....,0.0,32B_temp08_r2,42,4753,['astropy/wcs/wcsapi/wrappers/tests/test_slice...,0.020332,False


In [ ]:
def accuracy_topkp2p_then_max_yesprob(df, k=3):
    """
    For each group (docker_images):
       - Create a new column 'p2p_rate_len' as the length of the list in 'p2p_rate'.
       - Sort by 'p2p_rate_len' descending.
       - Take the top k rows.
       - From those, pick the row with the highest avg_yes_prob.
       - final_correct = that row's gt_correct (0 or 1).

    Overall accuracy = mean of final_correct across groups.
    """
    # Create a new column with the length of the p2p_rate list
    df = df.copy()  # To avoid modifying the original DataFrame
    df['p2p_rate_len'] = df['p2p_rate'].apply(len)
    
    grouped = df.groupby("docker_images")
    final_corrects = []

    for docker_image, group in grouped:
        # Sort by p2p_rate_len descending and take the top k rows
        topk = group.sort_values(by="p2p_rate_len", ascending=False).head(k)
        
        # Among the top k, select the row with the highest avg_yes_prob
        chosen_row = topk.loc[topk["avg_yes_prob"].idxmax()]
        
        # Record this group's correctness (0 or 1)
        final_corrects.append(chosen_row["gt_correct"])

    # Overall accuracy is the mean of the final correctness values across groups
    if not final_corrects:
        return 0.0
    return sum(final_corrects) / len(final_corrects)

In [44]:
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifiernew-32B_32k-v3.csv")


In [56]:
df

,docker_images,avg_yes_prob,gt_correct,predicted,correct_prediction
0,slimshetty/swebench-verified:sweb.eval.x86_64....,0.679179,1,1,1
1,slimshetty/swebench-verified:sweb.eval.x86_64....,0.777300,1,1,1
2,slimshetty/swebench-verified:sweb.eval.x86_64....,0.622459,1,1,1
3,slimshetty/swebench-verified:sweb.eval.x86_64....,0.851953,1,1,1
4,slimshetty/swebench-verified:sweb.eval.x86_64....,0.531209,1,1,1
...,...,...,...,...,...
4705,slimshetty/swebench-verified:sweb.eval.x86_64....,0.164516,0,0,1
4706,slimshetty/swebench-verified:sweb.eval.x86_64....,0.531209,0,1,0
4707,slimshetty/swebench-verified:sweb.eval.x86_64....,0.025957,0,0,1
4708,slimshetty/swebench-verified:sweb.eval.x86_64....,0.245085,1,0,0


In [58]:
topk_probs

docker_images
slimshetty/swebench-verified:sweb.eval.x86_64.astropy__astropy-12907    [0.3486451353339457, 0.3486451217982255, 0.320...
slimshetty/swebench-verified:sweb.eval.x86_64.astropy__astropy-13033    [0.7310585903489891, 0.731058584489497, 0.6513...
slimshetty/swebench-verified:sweb.eval.x86_64.astropy__astropy-13236    [0.6513548782017746, 0.6224593312018546, 0.592...
slimshetty/swebench-verified:sweb.eval.x86_64.astropy__astropy-13398    [0.7310585786300049, 0.5926665999540698, 0.592...
slimshetty/swebench-verified:sweb.eval.x86_64.astropy__astropy-13453    [0.3775406758018018, 0.2942149783515162, 0.268...
                                                                                              ...                        
slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-24213        [0.8354835432479661, 0.8175744828610095, 0.817...
slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-24443        [0.8354835411997897, 0.798186787341014, 0.7772...
slimshetty

In [13]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("./src/r2e_edits/agenthub/train/results_traj_verifier-14B-v2.csv")

best_overall_accuracy = 0.0
best_point = 0  # The number of trajectories at which the best performance was observed
intermediate_results = []  # List to store intermediate results (i, overall_accuracy, best_accuracy_so_far)

# Iterate over intermediate points
for i in range(1, len(df) + 1, 500):
    # Use the first i rows
    current_df = df.iloc[:i]
    
    # Group by 'docker_images' and select the row with the maximum avg_yes_prob for each group
    aggregated = current_df.groupby("docker_images", as_index=False).apply(
        lambda group: group.loc[group["avg_yes_prob"].idxmax()]
    ).reset_index(drop=True)
    
    # Calculate the overall accuracy (mean of gt_correct)
    overall_accuracy = aggregated["gt_correct"].mean()
    
    # Update the best performance so far if needed
    if overall_accuracy > best_overall_accuracy:
        best_overall_accuracy = overall_accuracy
        best_point = i
        
    # Save intermediate result
    intermediate_results.append((i, overall_accuracy, best_overall_accuracy))

print("Total number of trajectories:", len(df))
print("Best Aggregation Performance (Overall Accuracy):", best_overall_accuracy)
print("Achieved at intermediate point (number of trajectories):", best_point)

/tmp/ipykernel_2700259/94975078.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated = current_df.groupby("docker_images", as_index=False).apply(
/tmp/ipykernel_2700259/94975078.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated = current_df.groupby("docker_images", as_index=False).apply(
/tmp/ipykernel_2700259/94975078.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on th

Total number of trajectories: 4127
Best Aggregation Performance (Overall Accuracy): 0.39478957915831664
Achieved at intermediate point (number of trajectories): 3001


/tmp/ipykernel_2700259/94975078.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated = current_df.groupby("docker_images", as_index=False).apply(


In [17]:
import litellm

In [56]:
messages = swe_verifier_traj_ds['train'][10]['messages'][:2]

In [57]:
llm_name = 'hosted_vllm//data/home/jaskirats/project/r2e/r2e-edits-internal/LLaMA-Factory/saves/qwen25coder-14b-instruct-verifier-sonnet_32b_gpt4o_combined_32k_verifier_ep2-32k-v1'
llm_name = 'hosted_vllm//data/home/jaskirats/project/r2e/r2e-edits-internal/LLaMA-Factory/saves/qwen25coder-14b-instruct-verifier-sonnet_32b_gpt4o_combined_32k_verifiernew_reasoning_ep2-32k-v1'

num_samples = 1
temperature = 0.2
response = litellm.completion(
        model=llm_name,
        tools=[],
        messages=messages,
        n=num_samples,
        function_call=None,
        tool_choice="none",
        timeout=60,
        api_key=None,
        temperature=temperature,
        api_base="http://localhost:8000/v1",
        vertex_ai_project="r2eg-441800",
        vertex_ai_location="europe-west1",
        logprobs=True,
        top_logprobs=20,
    )

In [58]:
response

ModelResponse(id='chatcmpl-7474fdde0dde40bd84604b342cd9c100', created=1740547325, model='hosted_vllm//data/home/jaskirats/project/r2e/r2e-edits-internal/LLaMA-Factory/saves/qwen25coder-14b-instruct-verifier-sonnet_32b_gpt4o_combined_32k_verifiernew_reasoning_ep2-32k-v1', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='<judgement>YES</judgement>', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None, 'reasoning_content': None}, refusal=None, reasoning_content=None), logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='<', bytes=[60], logprob=0.0, top_logprobs=[TopLogprob(token='<', bytes=[60], logprob=0.0), TopLogprob(token='<j', bytes=[60, 106], logprob=-87.41463470458984), TopLogprob(token='</', bytes=[60, 47], logprob=-97.71707916259766), TopLogprob(token='jud', bytes=[106, 117, 100], logprob=-101.46341705322266), TopLogprob(token='<n', bytes=[60, 110]

In [59]:
response.choices[0].message.content

'<judgement>YES</judgement>'

In [60]:
response.choices[0].logprobs.content[4].top_logprobs

[TopLogprob(token='YES', bytes=[89, 69, 83], logprob=-0.07907514274120331),
 TopLogprob(token='NO', bytes=[78, 79], logprob=-2.5766336917877197),
 TopLogprob(token=' NO', bytes=[32, 78, 79], logprob=-55.025413513183594),
 TopLogprob(token=' YES', bytes=[32, 89, 69, 83], logprob=-57.05467987060547),
 TopLogprob(token='yes', bytes=[121, 101, 115], logprob=-58.14736557006836),
 TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-59.552242279052734),
 TopLogprob(token='.NO', bytes=[46, 78, 79], logprob=-63.9229736328125),
 TopLogprob(token='=YES', bytes=[61, 89, 69, 83], logprob=-65.79615020751953),
 TopLogprob(token='no', bytes=[110, 111], logprob=-65.79615020751953),
 TopLogprob(token='_NO', bytes=[95, 78, 79], logprob=-67.35712432861328),
 TopLogprob(token=':NO', bytes=[58, 78, 79], logprob=-67.98151397705078),
 TopLogprob(token='NOT', bytes=[78, 79, 84], logprob=-68.29370880126953),
 TopLogprob(token='Y', bytes=[89], logprob=-70.01078033447266),
 TopLogprob(token=' yes', bytes=[32, 

In [75]:
i=0
tokens = [x["token"] for x in response.choices[i].logprobs["content"]]
logits = [x["logprob"] for x in response.choices[i].logprobs["content"]]
# print (response.choices[0].logprobs.content[0].top_logprobs)
all_logits = [{
        lp.token: lp.logprob
        for lp in response.choices[0].logprobs.content[4].top_logprobs
    }]

In [76]:
all_logits

[{'YES': -0.07907514274120331,
  'NO': -2.5766336917877197,
  ' NO': -55.025413513183594,
  ' YES': -57.05467987060547,
  'yes': -58.14736557006836,
  'Yes': -59.552242279052734,
  '.NO': -63.9229736328125,
  '=YES': -65.79615020751953,
  'no': -65.79615020751953,
  '_NO': -67.35712432861328,
  ':NO': -67.98151397705078,
  'NOT': -68.29370880126953,
  'Y': -70.01078033447266,
  ' yes': -70.3229751586914,
  '(NO': -70.47907257080078,
  ':YES': -71.25955963134766,
  'TRUE': -71.25955963134766,
  '.YES': -71.72785186767578,
  'NE': -72.1961441040039,
  'FALSE': -72.43029022216797}]

In [77]:
tokens

['<', 'jud', 'gement', '>', 'YES', '</', 'jud', 'gement', '>', '']

In [79]:
j=4
try:
    k = tokens.index("YES", j)
except ValueError:
    try:
        k = tokens.index("NO", j)
    except ValueError:
        # try:
        #     k = tokens.index("YES", j)
        # except ValueError:
        #     try:
        #         k = tokens.index("NO", j)
        #     except ValueError:
        #         k = -1
        #         yes_probs.append(-1)
        #         continue
        k = -1
        yes_probs.append(-1)

k=0
# print(all_logits[k])
p_yes = all_logits[k].get("YES", -10000)
p_no = all_logits[k].get("NO", -10000)
# print(p_yes, p_no)
yes_probs = np.exp(p_yes) / (np.exp(p_yes) + np.exp(p_no))
# print(yes_probs[-1])
yes_probs

0.9239704878601753

In [67]:
all_logits = {
    lp.token: lp.logprob
    for lp in response.choices[0].logprobs.content[0].top_logprobs
}

In [68]:
all_logits

{'NO': -0.07907514274120331, 'YES': -2.5766336917877197}

# Debug

In [11]:
base_image = 'namanjain12/sympynew'
# base_image = 'namanjain12/pandasnew'
base_image = 'slimshetty/swebench-verified'
# base_image = 'namanjain12/coveragepy_final'

tags = fetch_docker_tags(base_image)
docker_image_list = [f'{base_image}:{x['name']}' for x in tags]

https://hub.docker.com/v2/repositories/slimshetty/swebench-verified/tags?page_size=50&page=11 not found


In [14]:
len(docker_image_list), docker_image_list

(500,
 ['slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-24562',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-24661',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-24539',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-24213',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-24443',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-23950',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-24066',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-23824',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-23534',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-23413',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-23262',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-22914',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-22714',
  'slimshetty/swebench-verified:sweb.eval.x86_64.sympy__sympy-22456',
  'slimshetty/

In [13]:
from datasets import load_dataset
ds = load_dataset("manishs/SWE-bench_Verified_augmented")
ds

DatasetDict({
    test: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit', 'parsed_commit', 'run_tests'],
        num_rows: 500
    })
})

In [16]:
# 3) Build a lookup from instance_id to docker_image
#    According to your note, you have `instance_id = docker_image.split('.')[-1]`
#    so we can invert this to map instance_id -> docker_image
docker_map = {img.split('.')[-1]: img for img in docker_image_list}

# 4) Define a function that will add the `docker_image` field to each row
def add_docker_image(example):
    # If there is no matching Docker image, you can insert a placeholder or None
    return {"docker_image": docker_map.get(example["instance_id"], None)}

# 5) Apply the function to the test split (or any relevant split in the ds dict)
ds_new = ds.map(add_docker_image)

# ds_new now contains an additional column "docker_image".
# You can inspect it:
ds_new

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit', 'parsed_commit', 'run_tests', 'docker_image'],
        num_rows: 500
    })
})

In [17]:
ds_new.push_to_hub('r2e-edits/swebench-verified-v1')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/r2e-edits/swebench-verified-v1/commit/1fe83d7d3cb55a5eac714155f360614b3b7c2ad2', commit_message='Upload dataset', commit_description='', oid='1fe83d7d3cb55a5eac714155f360614b3b7c2ad2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/r2e-edits/swebench-verified-v1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='r2e-edits/swebench-verified-v1'), pr_revision=None, pr_num=None)